In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# Generación para 365 días - TMY (considerado fecha de 2026)

Por simpliciadad vamos a mantener los factores d eponderación constantes e iguales a los del año 2023 , último año analizado. Se podrían actaulizar con las curvas de potencias, epro influye poco

## 1. Cargar los datos (El TMY se le ha puesto la fecha del 2026)

In [4]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Directorios de datos
directorio_13 = PARENT_DIR / "13. Mallado Potencia Instalada"
directorio_3 = PARENT_DIR / "03. Datos_Climaticos_TMY"

# df_inst_coord = pd.read_excel(f"{directorio_3}/PRETOR_REDUCIDO_Coord.xlsx", engine="openpyxl")
df_mall_pot = pd.read_csv(f"{directorio_13}/MALLADO_Potencia.csv", sep=',', encoding='latin-1')
df_mall_pob = pd.read_csv(f"{directorio_13}/MALLADO_Población.csv", sep=',', encoding='latin-1')
df_mall_clima = pd.read_csv(f"{directorio_3}/TMY_daily_agg_2026.csv", sep=',', encoding='latin-1')

# Tomamos los mallados de potencias de referencia como los del año 2023, y la de población constante que es la del 2024
df_mall_pot_ref = df_mall_pot.loc[df_mall_pot["Year"] == 2023].copy()
df_mall_pot_ref["Year"]=2026 # Cambiamos el año para que sea el que queremos

## 2. Introducir la key en clima

In [6]:
# Obtenemos las tres columnas de tiempo: año, mes, día
df_mall_clima['date_utc'] = pd.to_datetime(df_mall_clima['date_utc'], errors='coerce')

# Extrae año, mes (1–12) y día (1–31)
df_mall_clima['Year'] = df_mall_clima['date_utc'].dt.year
df_mall_clima['Month'] = df_mall_clima['date_utc'].dt.month
df_mall_clima['Day'] = df_mall_clima['date_utc'].dt.day 

# Añadimos la columna cell_id a CLIMA mediante un merge con POBLACIÓN
# Cambiamos nombre columnas
df_mall_clima = df_mall_clima.rename(columns={'lat': 'LAT_CENTRO', 'lon': 'LON_CENTRO'})

dec = 4
factor = 10 ** dec

df_mall_clima['LAT_CENTRO'] = np.trunc(df_mall_clima["LAT_CENTRO"] * factor) / factor
df_mall_clima["LON_CENTRO"] = np.trunc(df_mall_clima["LON_CENTRO"] * factor) / factor

df_mall_pob['LAT_CENTRO'] = np.trunc(df_mall_pob["LAT_CENTRO"] * factor) / factor
df_mall_pob["LON_CENTRO"] = np.trunc(df_mall_pob["LON_CENTRO"] * factor) / factor

df_mall_clima = df_mall_clima.merge(
    df_mall_pob[['cell_id', 'LAT_CENTRO', 'LON_CENTRO']],
    on=['LAT_CENTRO', 'LON_CENTRO'],
    how='left'
)

# Hacemos merge de potencia instalada
df_mall_clima = df_mall_clima.merge(
    df_mall_pot[['cell_id', 'Pot_Solar_kW', 'Pot_Eolico_kW']],
    on=['cell_id'],
    how='left'
)
 
# Hacemos merge de población
df_mall_clima = df_mall_clima.merge(
    df_mall_pob[['cell_id', 'POBLACION_TOTAL']],
    on=['cell_id'],
    how='left'
)

#Elegimos columnas finales
columnas_finales = ['cell_id', 'Year', 'Month', 'Day', 'Temp_min_C', 'Temp_max_C', 'Temp_media_C', 'RadiacionGlobal_media_Wm2', 'Viento_media_ms', 'Pot_Solar_kW', 'Pot_Eolico_kW', 'POBLACION_TOTAL', 'LAT_CENTRO', 'LON_CENTRO']
df_mall_total=[]
df_mall_total = df_mall_clima[columnas_finales]


In [7]:
# Guardar resultados
df_mall_total.to_csv(f"{BASE_DIR}/MALLADO_TOTAL_TMY.csv", index=False)